<a href="https://colab.research.google.com/github/chudriana/Fuzzy_Data_Matching/blob/main/fuzzy_matching_NN%2Bsentence_transformers_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import re
from sklearn.neighbors import NearestNeighbors
# model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
payroll = '/content/drive/MyDrive/MAPPING MITRA ID - KSM/FUZZY MATCHING/payroll.csv'
mitra = '/content/drive/MyDrive/MAPPING MITRA ID - KSM/FUZZY MATCHING/mitra.csv'

In [ ]:
df_mitra = pd.read_csv(mitra)
df_payroll = pd.read_csv(payroll)

In [ ]:
def preprocess_name(name):
    if pd.isna(name):
        return ""
    name = str(name).lower()
    name = re.sub(r'\b(pt|cv)\b', '', name)
    name = re.sub(r'[^a-z0-9\s]', '', name)
    # name = re.sub(r'\s+', ' ', name).strip()
    return name

In [ ]:
df_payroll['mitra_name_clean'] = df_payroll['mitra'].apply(preprocess_name)
df_mitra['mitra_name_clean'] = df_mitra['mitra'].apply(preprocess_name)

In [ ]:
df_mitra['embedding'] = df_mitra['mitra_name_clean'].apply(lambda x: model.encode(x))
df_payroll['embedding'] = df_payroll['mitra_name_clean'].apply(lambda x: model.encode(x))


In [ ]:
def fuzzy_match_embedding(source_df, target_df, source_col='embedding', target_col='embedding', threshold=0.85):
    source_embeddings = np.vstack(source_df[source_col].values)
    target_embeddings = np.vstack(target_df[target_col].values)

    nn = NearestNeighbors(n_neighbors=1, metric='cosine')
    nn.fit(target_embeddings)
    distances, indices = nn.kneighbors(source_embeddings)

    similarity_scores = 1 - distances.flatten()

    match_results = []
    for i, score in enumerate(similarity_scores):
        if score >= threshold:
            matched_row = {
                'payroll_name': source_df.iloc[i]['mitra'],
                'payroll_name_clean': source_df.iloc[i]['mitra_name_clean'],
                'matched_name': target_df.iloc[indices[i][0]]['mitra'],
                'matched_name_clean': target_df.iloc[indices[i][0]]['mitra_name_clean'],
                'mitra_id': target_df.iloc[indices[i][0]]['mitra_id'],
                # 'parent_id': target_df.iloc[indices[i][0]]['parent_id'],
                'similarity': score
            }
            match_results.append(matched_row)

    return pd.DataFrame(match_results)


In [ ]:
matched_df = fuzzy_match_embedding(df_payroll, df_mitra, threshold=0.85)
matched_df.head()

,payroll_name,payroll_name_clean,matched_name,matched_name_clean,mitra_id,similarity
0,Garuda Indonesia,garuda indonesia,Garuda Indonesia,garuda indonesia,M0089,1.000000
1,KAI (Kereta Api Indonesia),kai kereta api indonesia,Kereta Api Indonesia (KAI),kereta api indonesia kai,M0095,0.974167
2,Astra Honda Motor,astra honda motor,Toyota Astra Motor,toyota astra motor,M0050,0.874626
3,KPC (Kaltim Prima Coal),kpc kaltim prima coal,PT Kaltim Prima Coal,kaltim prima coal,M0028,0.864578
4,Ramayana,ramayana,Ramayana,ramayana,M0075,1.000000


In [ ]:
matched_df.to_csv('/content/drive/MyDrive/MAPPING MITRA ID - KSM/FUZZY MATCHING/matched_mitra_id_all-mpnet-base-v2.csv', index=False)